In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned MentalBERT model and tokenizer to prediction case
mentalbert_model_path = "fine_tuned_mentalbert_cause_classifier" 
mentalbert_tokenizer = AutoTokenizer.from_pretrained(mentalbert_model_path)
mentalbert_model = AutoModelForSequenceClassification.from_pretrained(mentalbert_model_path)
mentalbert_model = mentalbert_model.to(device)

# Load the fine-tuned RoBERTa model and tokenizer to generate predicted label
roberta_model_path = "fine_tuned_roberta_model"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_path)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_path)
roberta_model = roberta_model.to(device)

# Define label mappings
mentalbert_id2label = {
    0: "Health Issues",
    1: "Relationship Issues",
    2: "Financial Stress",
    3: "Workplace Stress",
    4: "Social Isolation",
    5: "No Issues"
}

roberta_id2label = {
    0: "Anxiety",
    1: "Bipolar",
    2: "Depression",
    3: "Normal",
    4: "Personality disorder",
    5: "Stress"
}

# Predict cause using MentalBERT
def predict_cause(statement):
    # Tokenize the input
    inputs = mentalbert_tokenizer(statement, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model
    
    # Perform prediction
    with torch.no_grad():
        outputs = mentalbert_model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    
    # Return the predicted cause
    cause = mentalbert_id2label[predicted_label]
    return cause

# Predict label using RoBERTa
def predict_label(statement):
    # Tokenize the input
    inputs = roberta_tokenizer(statement, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model
    
    # Perform prediction
    with torch.no_grad():
        outputs = roberta_model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    
    # Return the predicted label
    label = roberta_id2label[predicted_label]
    return label

# Initialize VADER SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Calculate urgency score based on VADER compound score
def calculate_urgency(statement):
    sentiment_scores = analyzer.polarity_scores(statement)
    compound_score = sentiment_scores['compound']
    
    # Normalize compound score to 1-10 scale and round to 2 decimal places
    urgency_score = round((compound_score + 1) * 5, 2)
    return urgency_score

# Generate Results
def analyze_statement(statement):
    predicted_label = predict_label(statement)
    predicted_cause = predict_cause(statement)
    
    if predicted_cause == "No Issues":
        urgency_score = "NA"
        urgency_level = "Low"
        flag_for_assistance = "No"
    else:
        urgency_score = calculate_urgency(statement)
        if urgency_score >= 7:
            urgency_level = "High"
            flag_for_assistance = "Yes"
        elif 4 <= urgency_score < 7:
            urgency_level = "Moderate"
            flag_for_assistance = "No"
        else:
            urgency_level = "Low"
            flag_for_assistance = "No"
    return {
        "Statement": statement,
        "Predicted Label": predicted_label,
        "Predicted Cause": predicted_cause,
        "Urgency Score": urgency_score,
        "Urgency Level": urgency_level,
        "Flag for Assistance": flag_for_assistance
    }

# Example statements
examples = [
    "Strange. I don't have work today and I have a bit of free time so I can read shrill novels but... it feels weird. I've been nervous about checking the google calendar just in case if I read it wrong. But it's still weird, how about this, like you should be looking for a job to get rid of this feeling.",
    "I dont like going out these days. No regrets or grudges/angry at things that have passed, and not worrying too much about the future, that's true serenity.",
    "Some days I'm very restless when I want to sleep. I often black out and faint",
    "I am feeling great today. Everything is wonderful!"
]

# Results
results = [analyze_statement(statement) for statement in examples]
df = pd.DataFrame(results)
print(df)

/home/fall2023/pg2360/miniconda3/envs/mental_health_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                           Statement Predicted Label  \
0  Strange. I don't have work today and I have a ...         Anxiety   
1  I dont like going out these days. No regrets o...      Depression   
2  Some days I'm very restless when I want to sle...         Anxiety   
3  I am feeling great today. Everything is wonder...          Normal   

    Predicted Cause Urgency Score Urgency Level Flag for Assistance  
0  Workplace Stress          2.37           Low                  No  
1  Social Isolation          7.95          High                 Yes  
2     Health Issues          3.64           Low                  No  
3         No Issues            NA           Low                  No  
